# About this note
- pytorch
- VGG16
- Image classification

## Library

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image

import torch
from torchvision import models,transforms
import torchvision
print('Pytorch ver:',torch.__version__)
print('torchvison ver:',torchvision.__version__)

Pytorch ver: 1.7.1
torchvison ver: 0.8.2


In [5]:
# config
pretrained = True
# vgg16
model = models.vgg16(pretrained=pretrained)

# 推論モード
model.eval()
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/kazuki/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [01:09<00:00, 7.99MB/s]
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(i

In [ ]:
class BaseTransform():
    # preprocess

    def __init__(self,)